Model_Training

In [80]:
import pandas as pd

In [81]:
df=pd.read_csv('./data/germstone.csv')

In [82]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [83]:
df=df.drop(labels=['id'],axis=1)

In [84]:
x=df.drop(labels='price',axis=1)
y=df[['price']]


In [85]:
cat_cols = x.select_dtypes(include='object').columns
num_cols = x.select_dtypes(exclude='object').columns
num_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [86]:
cut_cat=['Fair','Good','Very Good','Premium','Ideal']
color_cat=["D","E","F","G","H","I","J"]
clarity_cat = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [87]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [88]:
num_pipeline=Pipeline(steps=[
  ('imputer',SimpleImputer(strategy='median')),
  ('scaler',StandardScaler())
])

In [89]:
cat_pipeline=Pipeline(steps=[
  ('imputer',SimpleImputer(strategy='most_frequent')),
  ('encoder',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat])),
  ('scaler',StandardScaler())
])

In [90]:
preprocessor=ColumnTransformer([
  ('num_pipeline',num_pipeline,num_cols),
  ('cat_pipeline',cat_pipeline,cat_cols)
])

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)

In [93]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [95]:
import numpy as np

In [96]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [97]:
def model_performance(test,pred):
  mse=mean_squared_error(test,pred)
  rmse=np.sqrt(mean_squared_error(test,pred))
  mae=mean_absolute_error(test,pred)
  r2_scor=r2_score(test,pred)
  return mse,rmse,mae,r2_scor

In [98]:
models={
  'Linear_regression':LinearRegression(),
  'Lasso':Lasso(),
  'ridge':Ridge(),
  'elastic':ElasticNet()
}

In [100]:
model_list=[]
r2_list=[]

In [103]:
for i in range(len(models)):
  model=list(models.values())[i]
  model.fit(x_train,y_train)
  y_pred=model.predict(x_test)
  mse,rmse,mae,r2_scoree=model_performance(y_test,y_pred)
  print(list(models.keys())[i])
  print('model performance')
  print('MSE',mse)
  print('RMSE',rmse)
  print('MAE',mae)
  print('r2_score',r2_scoree*100)
  print('*'*40)
  print('\n')

Linear_regression
model performance
MSE 1028002.7598132559
RMSE 1013.9047094344004
MAE 674.0255115796832
r2_score 93.68908248567512
****************************************


Lasso
model performance
MSE 1027949.4559693959
RMSE 1013.8784226767013
MAE 675.0716923362162
r2_score 93.68940971841704
****************************************


ridge
model performance
MSE 1028005.2293677655
RMSE 1013.9059272771639
MAE 674.0555800798253
r2_score 93.68906732505938
****************************************


elastic
model performance
MSE 2351365.382289642
RMSE 1533.4162456064048
MAE 1060.7368759154729
r2_score 85.56494831165182
****************************************


